In [27]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector
import requests
import json
import cv2

In [29]:
mydb = mysql.connector.connect(host="localhost", user="root", password="", database="garbage")
cur = mydb.cursor()

In [30]:
model = load_model('Models/model.h5')
labels = {0:'cardboard', 1:'glass', 2:'metal', 3:'paper', 4:'plastic', 5:'trash'}

In [31]:
cap = cv2.VideoCapture(0)
for i in range(20):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('Frames/'+str(i)+'.jpg',frame)
cap.release()
cv2.destroyAllWindows()

In [36]:
test_img = 'Test_Images/plastic62.jpg'
img = image.load_img(test_img, target_size = (300,300))
img = image.img_to_array(img, dtype=np.uint8)
img = np.array(img)/255.0
prediction = model.predict(img[np.newaxis, ...])

In [37]:
print("Probability:",np.max(prediction[0], axis=-1))
predicted_class = labels[np.argmax(prediction[0], axis=-1)]
print("Classified:",predicted_class,'\n')

Probability: 0.98131126
Classified: plastic 



In [38]:
import requests
import json

send_url = "http://api.ipstack.com/152.57.8.231?access_key=f65bd90bacc856ed8bfd891376f75e12"
geo_req = requests.get(send_url)
geo_json = json.loads(geo_req.text)
latitude = geo_json['latitude']
longitude = geo_json['longitude']
city = geo_json['city']

print(latitude, longitude, city)

19.095199584960938 74.74960327148438 Ahmadnagar


In [39]:
s = "INSERT INTO kachra (type, lat, longi, city) VALUES (%s, %s, %s, %s)"
data = (predicted_class, latitude, longitude, city,)
cur.execute(s, data)
mydb.commit()